In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from itertools import combinations_with_replacement

In [2]:
in_path = r'C:\Users\User\Desktop\project 1'
names = ['mpg', 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'model_year', 'origin', 'car_name']

In [3]:
data = pd.read_csv('auto-mpg.data', sep='\s+',header=None, names = names, na_values = ['?'])

In [4]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


# 1. missing values 
To see if there exists missing values

In [5]:
data.dtypes

mpg             float64
cylinders         int64
displacement    float64
horsepower      float64
weight          float64
acceleration    float64
model_year        int64
origin            int64
car_name         object
dtype: object

In [6]:
data.isnull().sum()

mpg             0
cylinders       0
displacement    0
horsepower      6
weight          0
acceleration    0
model_year      0
origin          0
car_name        0
dtype: int64

In [7]:
data.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
count,398.000000,398.000000,398.000000,392.000000,398.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050,1.572864
std,7.815984,1.701004,104.269838,38.491160,846.841774,2.757689,3.697627,0.802055
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.250000,75.000000,2223.750000,13.825000,73.000000,1.000000
50%,23.000000,4.000000,148.500000,93.500000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,126.000000,3608.000000,17.175000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


### Horsepower has 6 missing values, we need to impute the missing values; Here, we use the mean values to impute

In [8]:
data.loc[data['horsepower'].isnull(), 'horsepower'] = np.mean(data['horsepower'])

In [9]:
data.describe()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin
count,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000,398.000000
mean,23.514573,5.454774,193.425879,104.469388,2970.424623,15.568090,76.010050,1.572864
std,7.815984,1.701004,104.269838,38.199187,846.841774,2.757689,3.697627,0.802055
min,9.000000,3.000000,68.000000,46.000000,1613.000000,8.000000,70.000000,1.000000
25%,17.500000,4.000000,104.250000,76.000000,2223.750000,13.825000,73.000000,1.000000
50%,23.000000,4.000000,148.500000,95.000000,2803.500000,15.500000,76.000000,1.000000
75%,29.000000,8.000000,262.000000,125.000000,3608.000000,17.175000,79.000000,2.000000
max,46.600000,8.000000,455.000000,230.000000,5140.000000,24.800000,82.000000,3.000000


In [10]:
data.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,model_year,origin,car_name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino


# 2. Model

## Multiple linear regression implementation

In [11]:
def MSE(W,X,Y):
    r = X.dot(W)-Y
    return r.dot(r)/X.shape[0]

def linear_model(X, Y):
    pinv = np.linalg.pinv(X)
    w = pinv.dot(Y)
    return w

## Polynomial regression implementation 

In [12]:
### get the polynomial features 
def polynomial_features(X, degree):
    n_samples, n_features = np.shape(X)
    
    ### get the feature combinations 
    def feature_combinations(degree):
        combinations = [combinations_with_replacement(range(n_features), i) for i in range(0, degree + 1)]
        feature_combinations = [item for degree_features in combinations for item in degree_features]
        return feature_combinations

    feature_combinations = feature_combinations(degree)
    n_output_features = len(feature_combinations)
    X_new = np.empty((n_samples, n_output_features))

    for i, index_combs in enumerate(feature_combinations):
        X_new[:, i] = np.prod(X[:, index_combs], axis=1)

    return X_new    

def polynomial_model(X, Y, degree):
    X_new = polynomial_features(X, degree)
    pinv = np.linalg.pinv(X_new)
    w = pinv.dot(Y)
    return w

# 3. Training, validation and test dataset.

## Randomly split data into 60%, 20%, 20% as train data, validate data and test data.

In [13]:
def split_data(data, percent):
    idx = np.random.rand(len(data)) < percent
    train, test = data[idx], data[~idx]
    return train, test

In [14]:
train_val_data, test_data = split_data(data, 0.8)   # split data into train_val_data and test_data

In [15]:
train_data, val_data = split_data(data, 0.75)   # split data into train_val_data and test_data

# Multiple regression model results

In [16]:
### standardize data function
def standardlize(X):
    return (X-X.mean())/X.std()

# need only stardardlize independent variables
std_train_x= np.array(standardlize(train_data.iloc[:,1:8]))
std_val_x =  np.array(standardlize(val_data.iloc[:,1:8]))

### For non-standardlize data

In [17]:
### cross validataion : 10-folder #### 

### initialize a list to contain the errors
errors = []

for _ in range(10):
    ### randomly split data
    train_data, val_data = split_data(data, 0.75) 
    
    ### get the train data and validate data 
    train_y,train_x= np.array(train_data.iloc[:,0]), np.array(train_data.iloc[:,1:8])
    val_y, val_x = np.array(val_data.iloc[:,0]), np.array(val_data.iloc[:,1:8])
    
    #### add ones to X
    ones = np.ones([train_x.shape[0],1])
    train_x = np.concatenate((ones,train_x),axis=1)
    ones = np.ones([val_x.shape[0],1])
    val_x = np.concatenate((ones,val_x),axis=1)   
    
    ### run linear model, and calculate the mean square error of traning data
    w_train = linear_model(train_x, train_y)
    mse_train = MSE(w_train, train_x, train_y)
    mse_val = MSE(w_train, val_x, val_y)
    errors.append([mse_train, mse_val])

df = pd.DataFrame(errors, columns=['Train Error', 'Validate Error']) 
df.loc['mean'] = df.mean()
df

,Train Error,Validate Error
0,10.470497,12.517295
1,10.572050,12.044433
2,10.683734,11.742135
3,10.703402,11.831405
4,11.116932,10.603115
5,10.656534,12.006686
6,10.091883,14.084086
7,10.454142,12.811889
8,10.297254,13.491302
9,11.596979,9.383135


### For standardlize data

In [18]:
### cross validataion : 10-folder #### 

### initialize a list to contain the errors
errors = []

for _ in range(10):
    ### randomly split data
    train_data, val_data = split_data(data, 0.75) 
    
    ### get the train data and validate data 
    train_y,train_x= np.array(train_data.iloc[:,0]), np.array(train_data.iloc[:,1:8])
    val_y, val_x = np.array(val_data.iloc[:,0]), np.array(val_data.iloc[:,1:8])
    
    ### standardize data and run the model again
    def standardlize(X):
        return (X-X.mean())/X.std()

    # need only stardardlize independent variables
    std_train_x= np.array(standardlize(train_data.iloc[:,1:8]))
    std_val_x =  np.array(standardlize(val_data.iloc[:,1:8]))
    
    #### add ones to X
    ones = np.ones([std_train_x.shape[0],1])
    train_x = np.concatenate((ones,std_train_x),axis=1)
    ones = np.ones([std_val_x.shape[0],1])
    val_x = np.concatenate((ones,std_val_x),axis=1)   
    
    ### run linear model, and calculate the mean square error of traning data
    w_train = linear_model(train_x, train_y)
    mse_train = MSE(w_train, train_x, train_y)
    mse_val = MSE(w_train, val_x, val_y)
    errors.append([mse_train, mse_val])

df = pd.DataFrame(errors, columns=['Train Error', 'Validate Error']) 
df.loc['mean'] = df.mean()
df

,Train Error,Validate Error
0,10.473070,12.247152
1,11.158863,10.703575
2,11.616883,10.787043
3,10.877081,11.467138
4,10.462748,12.968970
5,10.640846,13.964116
6,9.995528,15.285193
7,11.009249,11.251019
8,10.095513,13.890041
9,11.314285,10.339334


# Polynomial regression model results

### For non-standardlize data

In [19]:
### cross validataion : 10-folder #### 

def cv_unnorm(data, k):
    
    errors  = []

    for _ in range(10):
        ### randomly split data
        train_data, val_data = split_data(data, 0.75) 

        ### get the train data and validate data 
        train_y,train_x= np.array(train_data.iloc[:,0]), np.array(train_data.iloc[:,1:8])
        val_y, val_x = np.array(val_data.iloc[:,0]), np.array(val_data.iloc[:,1:8])
   
        x_train= polynomial_features(train_x, k)
        x_val = polynomial_features(val_x, k)
        w_train = linear_model(x_train, train_y)
        e1 = MSE(w_train, x_train, train_y)
        e2 = MSE(w_train, x_val, val_y)
        errors.append([e1,e2])
    return [sum(e) / len(e) for e in zip(*errors)]


### calculate the error for different degree
results = []
for i in range(2,5):
    error = cv_unnorm(data, i)
    results.append([i] + error)

df = pd.DataFrame(results, columns=['degree','Train Error', 'Validate Error']) 
df

,degree,Train Error,Validate Error
0,2,6.176001,8.631177
1,3,3.208025,39.805140
2,4,0.730928,10289.669918


### For standardlize data

In [20]:
### cross validataion : 10-folder #### 

def cv_norm(data, k):
    
    errors  = []

    for _ in range(10):
        ### randomly split data
        train_data, val_data = split_data(data, 0.75) 

        ### get the train data and validate data 
        train_y,train_x= np.array(train_data.iloc[:,0]), np.array(train_data.iloc[:,1:8])
        val_y, val_x = np.array(val_data.iloc[:,0]), np.array(val_data.iloc[:,1:8])
        
        # need only stardardlize independent variables
        std_train_x= np.array(standardlize(train_data.iloc[:,1:8]))
        std_val_x =  np.array(standardlize(val_data.iloc[:,1:8]))        

        x_train= polynomial_features(std_train_x, k)
        x_val = polynomial_features(std_val_x, k)
        w_train = linear_model(x_train, train_y)
        e1 = MSE(w_train, x_train, train_y)
        e2 = MSE(w_train, x_val, val_y)
        errors.append([e1,e2])
    return [sum(e) / len(e) for e in zip(*errors)]


### calculate the error for different degree
results = []
for i in range(2,5):
    error = cv_unnorm(data, i)
    results.append([i] + error)

df = pd.DataFrame(results, columns=['degree','Train Error', 'Validate Error']) 
df

,degree,Train Error,Validate Error
0,2,6.545872,7.090558
1,3,3.158253,42.186766
2,4,0.650605,13896.045454


# 5. Run Model with test data

In [25]:
std_test_y,std_test_x= np.array(test_data.iloc[:,0]), np.array(standardlize(test_data.iloc[:,1:8]))

In [26]:
# run ploynomial regression with degree 2
def polynomial_features(X, degree):
    n_samples, n_features = np.shape(X)
    
    ### get the feature combinations 
    def feature_combinations(degree, n_features):
        combinations = [combinations_with_replacement(range(n_features), i) for i in range(0, degree + 1)]
        feature_combinations = [item for degree_features in combinations for item in degree_features]
        return feature_combinations
    
    feature_combinations = feature_combinations(degree, n_features)
    n_output_features = len(feature_combinations)
    X_new = np.empty((n_samples, n_output_features))

    for i, index_combs in enumerate(feature_combinations):
        X_new[:, i] = np.prod(X[:, index_combs], axis=1)

    return X_new 

std_x_test = polynomial_features(std_test_x, 2)

std_w_test = linear_model(std_x_test, test_y)


print("The coefficients are: ")
print(std_w_test)
std_mse_test = MSE(std_w_test, std_x_test, test_y)
print("The MSE is: ")
print(std_mse_test)

The coefficients are: 
[20.7325049   0.91792511 -2.87602133 -4.1777755  -1.90320996 -1.6928225
  2.35708891  0.36041159 -2.8324095   2.78602326 -2.55719876  3.20195438
 -0.58055645 -1.49438509  0.71515494 -2.15131467  2.70677903  2.32936442
 -2.90068274  2.44014632 -0.19576898  3.48742671 -6.52268342  2.38442238
  1.05669705  0.15432897  1.61524334  1.56327615 -1.96539925 -0.95537229
 -0.3884302   1.05546428  0.24530555  0.63816693 -0.17707324 -0.54769831]
The MSE is: 
2.992622756010609
